<a href="https://colab.research.google.com/github/rymonk27-lab/MIS2800/blob/main/IC_Feb04_IC_MONK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IC (Feb 04) — Text Data in SQL (Student Version)

This in-class exercise practices the ideas from the Feb 04 slides:
- Text / character data and why it’s tricky
- `GROUP BY` + `COUNT()`
- `ORDER BY` (frequency vs alphabetical)
- Case sensitivity and `LOWER()` / `UPPER()`
- Spaces and `TRIM()`
- `LIKE` searches (and why they can be surprising)

**Instructions:** Run cells top → bottom. Write SQL inside the `query` blocks.


## 0) Setup (Run)
Creates a SQLite database `text_issues.db` and a table `product` with messy text.

In [13]:
!pip install --upgrade pandas ipython-sql prettytable==3.10.1

In [2]:
# Install the SQL extension (not in sources, but required for magics)
!pip install ipython-sql
print("ipython sql installed")
# lines of code will run below for the installation

ipython sql installed


In [3]:
# Load the SQL magic extension
%load_ext sql

In [4]:
import sqlite3, pandas as pd

In [5]:
# This creates a brand-new, empty database file named 'text_issues.db'
conn = sqlite3.connect('text_issues.db')
print("database created")

database created


In [6]:
%sql sqlite:///text_issues.db

In [7]:
# create a table called inventory_table
%%sql
CREATE TABLE inventory_table (
  id INTEGER PRIMARY KEY,
  category TEXT,
  note TEXT
) STRICT;

 * sqlite:///text_issues.db
Done.


[]

In [12]:
# complet the table creation
%%sql
INSERT INTO inventory_table (id, category, note)
VALUES
  (1,  "Apple",   "fresh"),
  (2,  "apple",   "fresh"),
  (3,  "APPLE",   "fresh"),
  (4,  "Banana",  "ripe"),
  (5,  "banana",  "ripe"),
  (6,  "banana ", "ripe"),
  (7,  " banana", "ripe"),
  (8,  "to-do",   "punctuation"),
  (9,  "to–do",   "punctuation"),
  (10, "",        "empty string"),
  (11, 'None',      "NULL value");

 * sqlite:///text_issues.db
(sqlite3.IntegrityError) UNIQUE constraint failed: inventory_table.id
[SQL: INSERT INTO inventory_table (id, category, note)
VALUES
  (1,  "Apple",   "fresh"),
  (2,  "apple",   "fresh"),
  (3,  "APPLE",   "fresh"),
  (4,  "Banana",  "ripe"),
  (5,  "banana",  "ripe"),
  (6,  "banana ", "ripe"),
  (7,  " banana", "ripe"),
  (8,  "to-do",   "punctuation"),
  (9,  "to–do",   "punctuation"),
  (10, "",        "empty string"),
  (11, 'None',      "NULL value");]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [9]:
%%sql
SELECT * FROM inventory_table;

 * sqlite:///text_issues.db
Done.


id,category,note
1,Apple,fresh
2,apple,fresh
3,APPLE,fresh
4,Banana,ripe
5,banana,ripe
6,banana,ripe
7,banana,ripe
8,to-do,punctuation
9,to–do,punctuation
10,,empty string


In [11]:
%%sql
SELECT * FROM inventory_table;

 * sqlite:///text_issues.db
Done.


id,category,note
1,Apple,fresh
2,apple,fresh
3,APPLE,fresh
4,Banana,ripe
5,banana,ripe
6,banana,ripe
7,banana,ripe
8,to-do,punctuation
9,to–do,punctuation
10,,empty string


## Part A — Quick Concept Check (We do it together)
1. Which values in `category` refer to the *same* category but look different?
2. What is the difference between an **empty string** (`''`) and **NULL**?
3. Why might punctuation cause matching problems?


#1. apple vs APPLE, 'APPLE' vs ' aPPLE'

#2. empty string is a space, null is no value.

#3. the machine will take different punctation as different values.

## Part B1 — Grouping and counting (raw text)
Which product categories have the most products, and how many products are in each category (sorted from most to least)

In [15]:
%%sql
Select category, count(category)
from inventory_table
group by category
order by count(category) DESC;




 * sqlite:///text_issues.db
Done.


category,count(category)
to–do,1
to-do,1
banana,1
banana,1
apple,1
None,1
Banana,1
Apple,1
APPLE,1
banana,1


## Part B2 — Fix grouping with LOWER() - individual one

In [16]:
%%sql
SELECT LOWER(category), COUNT(category)
FROM inventory_table
WHERE category IS NOT NULL
GROUP BY LOWER(category)
ORDER BY COUNT(category) DESC;

 * sqlite:///text_issues.db
Done.


LOWER(category),COUNT(category)
apple,3
banana,2
to–do,1
to-do,1
none,1
banana,1
banana,1
,1


## Part B3 — Fix spaces with TRIM() + LOWER()

In [17]:
%%sql
SELECT LOWER(TRIM(category)), COUNT(category)
FROM inventory_table
WHERE category IS NOT NULL
GROUP BY LOWER(TRIM(category))
ORDER BY COUNT(category) DESC;

 * sqlite:///text_issues.db
Done.


LOWER(TRIM(category)),COUNT(category)
banana,4
apple,3
to–do,1
to-do,1
none,1
,1


## Part C1 — Order by category (alphabetical)

In [18]:
%%sql
SELECT DISTINCT LOWER(TRIM(category))
FROM inventory_table
WHERE category IS NOT NULL AND category != ''
ORDER BY LOWER(TRIM(category));

 * sqlite:///text_issues.db
Done.


LOWER(TRIM(category))
apple
banana
none
to-do
to–do


## Part D1 — Case matters

In [19]:
%%sql
SELECT *
FROM inventory_table
WHERE category = 'apple';

 * sqlite:///text_issues.db
Done.


id,category,note
2,apple,fresh


## Part D2 — Case-insensitive match

In [20]:
%%sql
SELECT *
FROM inventory_table
WHERE LOWER(TRIM(category)) = 'apple';

 * sqlite:///text_issues.db
Done.


id,category,note
1,Apple,fresh
2,apple,fresh
3,APPLE,fresh


## Part D3 — Empty strings aren’t NULL

In [21]:
%%sql
SELECT *
FROM inventory_table
WHERE category = '';

 * sqlite:///text_issues.db
Done.


id,category,note
10,,empty string


## Part D4 — Find NULL categories

In [22]:
%%sql
SELECT *
FROM inventory_table
WHERE category IS NULL;

 * sqlite:///text_issues.db
Done.


id,category,note
